<a href="https://colab.research.google.com/github/HSS107048212/hoop4s/blob/main/%E5%90%88%E4%BD%9C%E7%A4%BE%E7%95%B6%E6%97%A5%E3%80%8C%E4%BE%9B%E6%87%89%E9%9C%80%E6%B1%82%E5%96%AE%E3%80%8D%E6%9F%A5%E8%A9%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

轉換後程式

In [1]:
# Installation and imports
!pip3 install pygsheets
!pip3 install gspread-dataframe==3.2.1
!pip3 install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: gspread-dataframe
    Found existing installation: gspread-dataframe 3.3.1
    Uninstalling gspread-dataframe-3.3.1:
      Successfully uninstalled gspread-dataframe-3.3.1


# SERVER 端程式

In [2]:
import os
import re
import smtplib
import pandas as pd
import gspread

from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Remaining code has been shortened for brevity...

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

import pygsheets

#HOME_PATH = '/home/iss/realplus/'
#HOME_PATH = '/home/iss/realplus/'
HOME_PATH = '/content/' # path of Colab for testing

# Authorization
gc = pygsheets.authorize(service_file=HOME_PATH+'orderdata-copyto-parttime-5c19ba8fcba4.json')

# Google sheet URL and authentication
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/15jbMd0cvO2OCcgxZiQJLG1jSR6rXyXn1mnl7SCoFoTM/edit?usp=sharing'
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("表單回應 1")

# Load data into dataframe
Order_data = pd.DataFrame(wks.get_all_records())

# Function to convert Chinese timestamp to a suitable format
def convert_timestamp(timestamp):
    return re.sub(r'(上午|下午)', '', timestamp)

Order_data['時間戳記'] = Order_data['時間戳記'].apply(convert_timestamp)
Order_data['日期'] = pd.to_datetime(Order_data['時間戳記']).dt.strftime('%Y%m%d')
Order_data['當日訂單數量'] = Order_data.groupby('日期').cumcount().add(1).apply(lambda x: f'{x:03d}')
Order_data['訂單編號'] = Order_data['日期'] + Order_data['當日訂單數量']

# Drop unnecessary columns
Order_data.drop(['日期', '當日訂單數量'], axis=1, inplace=True)

# Get today's date
today = datetime.today().date()

# Split timestamp column into date and time
Order_data['日期'] = Order_data['時間戳記'].apply(lambda x: x.split()[0])
Order_data['日期'] = pd.to_datetime(Order_data['日期'], format='%Y/%m/%d')
today_rows = Order_data[Order_data['日期'].dt.date == today]
Order_data = today_rows

# Load ProductID_data
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/10D0oXYr1kFF_EOCS_E2lpBJv52aFmQ9qVxG-evC0D7M/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("產品編號")
ProductID_data = pd.DataFrame(wks.get_all_records())

# Load Supplier_data
wks = sht.worksheet_by_title("供應商資訊")
Supplier_data = pd.DataFrame(wks.get_all_records())

deliver_columns = Order_data.columns[6:-1]
deliver = Order_data[deliver_columns]
column_names = deliver[:-1].columns.tolist()
column_names = column_names[:-1]

product_columns = column_names
result = {}

for product_column in product_columns:
    filtered_orders = deliver[deliver[product_column] != ""]["訂單編號"]
    result[product_column] = [filtered_orders.tolist()]

products_columns = Order_data.columns[6:-2]
Order_data = Order_data[products_columns]
product_quantities = Order_data.apply(pd.to_numeric, errors='coerce').fillna(0)
item_totals = product_quantities.sum().to_dict()

df = pd.DataFrame(ProductID_data, columns=["供應商", "產品全稱"])
selected_columns = df[["供應商", "產品全稱"]]
selected_columns['訂購數量'] = selected_columns['產品全稱'].map(item_totals).fillna(0)



# Assuming 'selected_columns' is your DataFrame
grouped = selected_columns.groupby("供應商")

# Load the billing template
template_path = HOME_PATH+'suppliers.xlsx'  # TODO: Set as the server path
template_wb = load_workbook(template_path)

# Create a directory to save the individual Excel files
today = datetime.now().strftime("%Y-%m-%d")
output_directory = f'{HOME_PATH}content/{today}/'  # TODO: Set as the server path
os.makedirs(output_directory, exist_ok=True)

# Set email sender credentials as environment variables
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "coop4s@nthu"

# Function to set cell formatting
def set_cell_formatting(sheet):
    font = Font(size=20)
    alignment = Alignment(horizontal='center', vertical='center')
    fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
    border = Border(
        left=Side(border_style='thick'),
        right=Side(border_style='thick'),
        top=Side(border_style='thick'),
        bottom=Side(border_style='thick')
    )

    sheet['A1'].font = font
    sheet['A1'].alignment = alignment
    sheet['A1'].fill = fill
    sheet['A2'].alignment = alignment

    for row in sheet.iter_rows():
        for cell in row:
            cell.border = border

# Iterate through each supplier group and save as separate Excel files
for group_name, group_data in grouped:
    # Create a new workbook for each group_name
    wb = Workbook()
    new_sheet = wb.active
    new_sheet.title = group_name

    # Setup column dimensions and merge cells
    new_sheet.column_dimensions['A'].width = 20
    new_sheet.column_dimensions['B'].width = 40
    new_sheet.column_dimensions['C'].width = 40
    new_sheet.merge_cells('A1:C1')
    new_sheet.merge_cells('A2:C2')

    set_cell_formatting(new_sheet)
    new_sheet.delete_rows(15, new_sheet.max_row)

    # Copy the contents of "供應商訂單" to the new sheet
    source_sheet = template_wb["供應商訂單"]
    for row_index, source_row in enumerate(source_sheet.iter_rows(), start=1):
        for column_index, cell in enumerate(source_row, start=1):
            new_sheet.cell(row=row_index, column=column_index, value=cell.value)

    # Calculate the starting row for data in the new sheet (B15 and C15)
    start_row = 15

    # Retrieve the supplier information based on group_name from 'Supplier_data'
    supplier_info = Supplier_data[Supplier_data["供應商名稱"] == group_name].iloc[0]

    # Write supplier-specific information to the new sheet
    new_sheet.cell(row=8, column=2, value=supplier_info["法人姓名"])
    new_sheet.cell(row=10, column=2, value=str(supplier_info["電話"]))
    new_sheet.cell(row=11, column=2, value=supplier_info["Email"])
    new_sheet.cell(row=9, column=2, value=supplier_info["地址"])
    new_sheet.cell(row=7, column=2, value=group_name)

    reminder=0
    remind_string="目前沒有社員訂購您的產品或服務。"
    # Write the group data to the Excel file starting from B15 and C15
    for index, row in group_data.iterrows():
        new_sheet.cell(row=start_row, column=2, value=row['產品全稱'])
        new_sheet.cell(row=start_row, column=3, value=row['訂購數量'])
        reminder += row['訂購數量']
        product_name = row['產品全稱']
        if product_name in result.keys():
            values = result[product_name]
            values = ''.join(map(str, values))
            new_sheet.cell(row=start_row, column=4, value=values)

        else:
            values = None  # 或者选择一个默认值
        #print(product_name)
        #print(values)

        start_row += 1

    if reminder != 0 :
      # Save the current workbook as an individual Excel file
      output_filename = os.path.join(output_directory, f"{today}{group_name}.xlsx")
      wb.save(output_filename)
      # Email settings
      sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
      sender_password = "jwsscmoiffdyukmm"
      recipient_email = str(supplier_info["Email"])  # 確保將電子郵件地址轉換為字符串
      #print(recipient_email)
      subject = "【十里方圓合作社】供應商產品訂購訂單通知。"
      message = MIMEMultipart()
      message["From"] = sender_email
      message["To"] = recipient_email
      message["Subject"] = subject

      body = "供應商您好，請檢閱產品訂購數量，若有問題，請您聯絡： 莊棋本 先生，電話：0927433990，E-mail：louie311@gmail.com，再次感謝您的配合。"
      message.attach(MIMEText(body, 'plain'))

      with open(output_filename, 'rb') as attachment:
          part = MIMEApplication(attachment.read(), Name=os.path.basename(output_filename))
      part['Content-Disposition'] = f'attachment; filename="{os.path.basename(output_filename)}"'
      message.attach(part)

      try:
          server = smtplib.SMTP('smtp.gmail.com', 587)  # Replace with your SMTP server details
          server.starttls()
          server.login(sender_email, sender_password)

          server.sendmail(sender_email, recipient_email, message.as_string())
          server.quit()

          print(f"Email sent to {recipient_email} with attached Excel file.")
      except smtplib.SMTPException as e:
          print(f"Email sending failed: {e}")
      except Exception as e:
          print(f"An error occurred: {e}")

      # Remove the default sheet (if exists)
      if 'Sheet' in template_wb.sheetnames:
          template_wb.remove(template_wb['Sheet'])

      # Print completion message
      print(f"Excel files have been created for each supplier and saved to the '{output_directory}' directory.")

# Get the list of files in the output directory
files = os.listdir(output_directory)

# Load your service account key JSON file and initialize Google Drive API client
credentials = service_account.Credentials.from_service_account_file(
    HOME_PATH+'orderdata-copyto-parttime-bbb4e17511c7.json',
    scopes=['https://www.googleapis.com/auth/drive']
)
drive_service = build('drive', 'v3', credentials=credentials)

# Function to upload files to a specific Google Drive folder
def upload_files_to_drive(folder_id, files, today):
    for file_name in files:
        # Specify the file path and create a media upload object
        file_path = f'{HOME_PATH}content/{today}/{file_name}'  # Please modify the path accordingly
        media = MediaFileUpload(file_path, mimetype='application/octet-stream')

        # Upload the file to the target folder
        file_metadata = {
            'name': file_name,
            'parents': [folder_id]
        }
        uploaded_file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id'
        ).execute()

        print(f'文件 "{file_name}" 已成功上傳到 Google Drive 文件夾，文件 ID：{uploaded_file.get("id")}')

# Specify the target folder IDs (obtained from the provided URLs) and upload the files
folder_id1 = '1rVzIKeKaMupnTm0CJlTb6Lo8XfZQL2yF'
upload_files_to_drive(folder_id1, files, today)

folder_id2 = '1w0-h2ZLJH7CbkEmgE8FBoEvk9UKlud7H'
upload_files_to_drive(folder_id2, files, today)


Email sent to wufengcn001@gmail.com  with attached Excel file.
Excel files have been created for each supplier and saved to the '/content/content/2023-10-17/' directory.
Email sent to wufengcn001@gmail.com  with attached Excel file.
Excel files have been created for each supplier and saved to the '/content/content/2023-10-17/' directory.
文件 "2023-10-17有限責任新竹縣馬武督咖啡生產合作社.xlsx" 已成功上傳到 Google Drive 文件夾，文件 ID：1ItVPxm8PpPoBBobs7QtBfcwKnv-qBUV6
文件 "2023-10-17富林農場.xlsx" 已成功上傳到 Google Drive 文件夾，文件 ID：1jdS9oAXHihVc1jp8WeoMIwIEXRgCnIcl
文件 "2023-10-17有限責任新竹縣馬武督咖啡生產合作社.xlsx" 已成功上傳到 Google Drive 文件夾，文件 ID：1JD4qW-ZKlh_Z4P5bgXZRThyONjsPSLHk
文件 "2023-10-17富林農場.xlsx" 已成功上傳到 Google Drive 文件夾，文件 ID：1Ap2h_aIdKgbsH_W65hZOrb9a0v8HX8os


In [ ]:
Order_data

,富林農場有機蓬萊米,富林農場有機蓬萊胚芽米,富林農場有機蓬萊糙米,豐禾子「高雄147」友善米,豐禾子「台中194」友善米 [第 1 列],赤柯山有機苦茶油,赤柯山台灣小果苦茶油,赤柯山有機轉型期印加果油,赤柯山印加果油隨身包,嫩馨香椿農場嫩馨香樁SOD超氧化物歧化酶酵素,...,有限責任新竹縣馬武督咖啡生產合作社水洗處理咖啡豆,有限責任新竹縣馬武督咖啡生產合作社蜜處理咖啡豆,有限責任新竹縣馬武督咖啡生產合作社日曬處理咖啡豆,有限責任新竹縣馬武督咖啡生產合作社厭氧蜜處理咖啡豆,有限責任新竹縣馬武督咖啡生產合作社厭氧水洗處理咖啡豆,有限責任新竹縣馬武督咖啡生產合作社薰衣草處理咖啡豆,有限責任新竹縣馬武督咖啡生產合作社濾掛咖啡10包,有限責任新竹縣馬武督咖啡生產合作社濾掛咖啡16包禮盒,有限責任新竹縣馬武督咖啡生產合作社濾掛咖啡20包禮盒,有限責任新竹縣馬武督咖啡生產合作社咖啡葉茶中淺焙75g
56,,,,1,0,,,,,,...,,,,,,,,,,


import os
import re
import smtplib
import pandas as pd
import gspread

from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Remaining code has been shortened for brevity...

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

import pygsheets

#HOME_PATH = '/home/iss/realplus/'
#HOME_PATH = '/home/iss/realplus/'
HOME_PATH = '/content/' # path of Colab for testing

# Authorization
gc = pygsheets.authorize(service_file=HOME_PATH+'orderdata-copyto-parttime-5c19ba8fcba4.json')

# Google sheet URL and authentication
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/15jbMd0cvO2OCcgxZiQJLG1jSR6rXyXn1mnl7SCoFoTM/edit?usp=sharing'
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("表單回應 1")

# Load data into dataframe
Order_data = pd.DataFrame(wks.get_all_records())

# Function to convert Chinese timestamp to a suitable format
def convert_timestamp(timestamp):
    return re.sub(r'(上午|下午)', '', timestamp)

Order_data['時間戳記'] = Order_data['時間戳記'].apply(convert_timestamp)
Order_data['日期'] = pd.to_datetime(Order_data['時間戳記']).dt.strftime('%Y%m%d')
Order_data['當日訂單數量'] = Order_data.groupby('日期').cumcount().add(1).apply(lambda x: f'{x:03d}')
Order_data['訂單編號'] = Order_data['日期'] + Order_data['當日訂單數量']

# Drop unnecessary columns
Order_data.drop(['日期', '當日訂單數量'], axis=1, inplace=True)

# Get today's date
today = datetime.today().date()

# Split timestamp column into date and time
Order_data['日期'] = Order_data['時間戳記'].apply(lambda x: x.split()[0])
Order_data['日期'] = pd.to_datetime(Order_data['日期'], format='%Y/%m/%d')
today_rows = Order_data[Order_data['日期'].dt.date == today]
Order_data = today_rows

# Load ProductID_data
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/10D0oXYr1kFF_EOCS_E2lpBJv52aFmQ9qVxG-evC0D7M/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("產品編號")
ProductID_data = pd.DataFrame(wks.get_all_records())

# Load Supplier_data
wks = sht.worksheet_by_title("供應商資訊")
Supplier_data = pd.DataFrame(wks.get_all_records())

deliver_columns = Order_data.columns[6:-1]
deliver = Order_data[deliver_columns]
column_names = deliver[:-1].columns.tolist()
column_names = column_names[:-1]

product_columns = column_names
result = {}

for product_column in product_columns:
    filtered_orders = deliver[deliver[product_column] != ""]["訂單編號"]
    result[product_column] = [filtered_orders.tolist()]

products_columns = Order_data.columns[6:-2]
Order_data = Order_data[products_columns]
product_quantities = Order_data.apply(pd.to_numeric, errors='coerce').fillna(0)
item_totals = product_quantities.sum().to_dict()

df = pd.DataFrame(ProductID_data, columns=["供應商", "產品全稱"])
selected_columns = df[["供應商", "產品全稱"]]
selected_columns['訂購數量'] = selected_columns['產品全稱'].map(item_totals).fillna(0)



# Assuming 'selected_columns' is your DataFrame
grouped = selected_columns.groupby("供應商")

# Load the billing template
template_path = HOME_PATH+'suppliers.xlsx'  # TODO: Set as the server path
template_wb = load_workbook(template_path)

# Create a directory to save the individual Excel files
today = datetime.now().strftime("%Y-%m-%d")
output_directory = f'{HOME_PATH}content/{today}/'  # TODO: Set as the server path
os.makedirs(output_directory, exist_ok=True)

# Set email sender credentials as environment variables
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "coop4s@nthu"

# Function to set cell formatting
def set_cell_formatting(sheet):
    font = Font(size=20)
    alignment = Alignment(horizontal='center', vertical='center')
    fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
    border = Border(
        left=Side(border_style='thick'),
        right=Side(border_style='thick'),
        top=Side(border_style='thick'),
        bottom=Side(border_style='thick')
    )

    sheet['A1'].font = font
    sheet['A1'].alignment = alignment
    sheet['A1'].fill = fill
    sheet['A2'].alignment = alignment

    for row in sheet.iter_rows():
        for cell in row:
            cell.border = border

# Iterate through each supplier group and save as separate Excel files
for group_name, group_data in grouped:
    # Create a new workbook for each group_name
    wb = Workbook()
    new_sheet = wb.active
    new_sheet.title = group_name

    # Setup column dimensions and merge cells
    new_sheet.column_dimensions['A'].width = 20
    new_sheet.column_dimensions['B'].width = 40
    new_sheet.column_dimensions['C'].width = 40
    new_sheet.merge_cells('A1:C1')
    new_sheet.merge_cells('A2:C2')

    set_cell_formatting(new_sheet)
    new_sheet.delete_rows(15, new_sheet.max_row)

    # Copy the contents of "供應商訂單" to the new sheet
    source_sheet = template_wb["供應商訂單"]
    for row_index, source_row in enumerate(source_sheet.iter_rows(), start=1):
        for column_index, cell in enumerate(source_row, start=1):
            new_sheet.cell(row=row_index, column=column_index, value=cell.value)

    # Calculate the starting row for data in the new sheet (B15 and C15)
    start_row = 15

    # Retrieve the supplier information based on group_name from 'Supplier_data'
    supplier_info = Supplier_data[Supplier_data["供應商名稱"] == group_name].iloc[0]

    # Write supplier-specific information to the new sheet
    new_sheet.cell(row=8, column=2, value=supplier_info["法人姓名"])
    new_sheet.cell(row=10, column=2, value=str(supplier_info["電話"]))
    new_sheet.cell(row=11, column=2, value=supplier_info["Email"])
    new_sheet.cell(row=9, column=2, value=supplier_info["地址"])
    new_sheet.cell(row=7, column=2, value=group_name)

    reminder=0
    remind_string="目前沒有社員訂購您的產品或服務。"
    # Write the group data to the Excel file starting from B15 and C15
    for index, row in group_data.iterrows():
        new_sheet.cell(row=start_row, column=2, value=row['產品全稱'])
        new_sheet.cell(row=start_row, column=3, value=row['訂購數量'])
        reminder += row['訂購數量']
        product_name = row['產品全稱']
        if product_name in result.keys():
            values = result[product_name]
            values = ''.join(map(str, values))
            new_sheet.cell(row=start_row, column=4, value=values)

        else:
            values = None  # 或者选择一个默认值
        #print(product_name)
        #print(values)

        start_row += 1

    if reminder == 0 :
      new_sheet.cell(row=13, column=2, value=remind_string)

    # Save the current workbook as an individual Excel file
    output_filename = os.path.join(output_directory, f"{today}{group_name}.xlsx")
    wb.save(output_filename)
    #backup_filename = os.path.join(backup_directory, f"{today}{group_name}.xlsx")
    #wb.save(backup_filename)

    # Email settings
    sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
    sender_password = "jwsscmoiffdyukmm"
    recipient_email = str(supplier_info["Email"])  # 確保將電子郵件地址轉換為字符串
    #print(recipient_email)
    subject = "【十里方圓合作社】供應商產品訂購訂單通知。"
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recipient_email
    message["Subject"] = subject

    body = "供應商您好，請檢閱產品訂購數量，若有問題，請您聯絡： 莊棋本 先生，電話：0927433990，E-mail：louie311@gmail.com，再次感謝您的配合。"
    message.attach(MIMEText(body, 'plain'))

    with open(output_filename, 'rb') as attachment:
        part = MIMEApplication(attachment.read(), Name=os.path.basename(output_filename))
    part['Content-Disposition'] = f'attachment; filename="{os.path.basename(output_filename)}"'
    message.attach(part)

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)  # Replace with your SMTP server details
        server.starttls()
        server.login(sender_email, sender_password)

        server.sendmail(sender_email, recipient_email, message.as_string())
        server.quit()

        print(f"Email sent to {recipient_email} with attached Excel file.")
    except smtplib.SMTPException as e:
        print(f"Email sending failed: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

    # Remove the default sheet (if exists)
    if 'Sheet' in template_wb.sheetnames:
        template_wb.remove(template_wb['Sheet'])

    # Print completion message
    print(f"Excel files have been created for each supplier and saved to the '{output_directory}' directory.")

# Get the list of files in the output directory
files = os.listdir(output_directory)

# Load your service account key JSON file and initialize Google Drive API client
credentials = service_account.Credentials.from_service_account_file(
    HOME_PATH+'orderdata-copyto-parttime-bbb4e17511c7.json',
    scopes=['https://www.googleapis.com/auth/drive']
)
drive_service = build('drive', 'v3', credentials=credentials)

# Function to upload files to a specific Google Drive folder
def upload_files_to_drive(folder_id, files, today):
    for file_name in files:
        # Specify the file path and create a media upload object
        file_path = f'{HOME_PATH}content/{today}/{file_name}'  # Please modify the path accordingly
        media = MediaFileUpload(file_path, mimetype='application/octet-stream')

        # Upload the file to the target folder
        file_metadata = {
            'name': file_name,
            'parents': [folder_id]
        }
        uploaded_file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id'
        ).execute()

        print(f'文件 "{file_name}" 已成功上傳到 Google Drive 文件夾，文件 ID：{uploaded_file.get("id")}')

# Specify the target folder IDs (obtained from the provided URLs) and upload the files
folder_id1 = '1rVzIKeKaMupnTm0CJlTb6Lo8XfZQL2yF'
upload_files_to_drive(folder_id1, files, today)

folder_id2 = '1w0-h2ZLJH7CbkEmgE8FBoEvk9UKlud7H'
upload_files_to_drive(folder_id2, files, today)


# 原始程式

In [ ]:
!pip3 install pygsheets
!pip3 install gspread-dataframe==3.2.1
import pygsheets
gc = pygsheets.authorize(service_file='/content/orderdata-copyto-parttime-5c19ba8fcba4.json') # Gild學長後續請將.json金鑰設定在local路徑
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd
# google sheet 網址與認證
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/15jbMd0cvO2OCcgxZiQJLG1jSR6rXyXn1mnl7SCoFoTM/edit?usp=sharing'
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("表單回應 1")
#讀取成 df
Order_data = pd.DataFrame(wks.get_all_records())
import pandas as pd
import re


# 定义一个函数来将中文时间戳转换为适合解析的格式
def convert_timestamp(timestamp):
    # 使用正则表达式将中文时间戳中的"上午"或"下午"替换为空格，同时保留原时间
    converted_timestamp = re.sub(r'(上午|下午)', '', timestamp)
    return converted_timestamp

# 对时间戳列应用转换函数
Order_data['時間戳記'] = Order_data['時間戳記'].apply(convert_timestamp)

# 将时间戳列转换为日期格式，以便按日期分组
Order_data['日期'] = pd.to_datetime(Order_data['時間戳記']).dt.strftime('%Y%m%d')

# 使用groupby对每天的订单进行累加计数，以三位数的格式表示
Order_data['當日訂單數量'] = Order_data.groupby('日期').cumcount().add(1).apply(lambda x: f'{x:03d}')

# 生成訂單編號
Order_data['訂單編號'] = Order_data['日期'] + Order_data['當日訂單數量']

# 删除不需要的列
Order_data.drop(['日期', '當日訂單數量'], axis=1, inplace=True)

from dateutil.parser import parse
from datetime import datetime
# 獲取今天的日期
today = datetime.today().date()

# 將時間戳記列拆分為日期和時間
Order_data['日期'] = Order_data['時間戳記'].apply(lambda x: x.split()[0])

# 將日期字符串轉換為日期對象
Order_data['日期'] = pd.to_datetime(Order_data['日期'], format='%Y/%m/%d')

# 挑選出日期與今天相同的行
today_rows = Order_data[Order_data['日期'].dt.date == today]
Order_data=today_rows

Order_data

# google sheet 網址與認證
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/10D0oXYr1kFF_EOCS_E2lpBJv52aFmQ9qVxG-evC0D7M/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("產品編號")
#讀取成 df
ProductID_data = pd.DataFrame(wks.get_all_records())

# 打印DataFrame
ProductID_data

# google sheet 網址與認證
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/10D0oXYr1kFF_EOCS_E2lpBJv52aFmQ9qVxG-evC0D7M/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("供應商資訊")
#讀取成 df
Supplier_data = pd.DataFrame(wks.get_all_records())
Supplier_data

deliver_columns = Order_data.columns[6:-1]
deliver = Order_data[deliver_columns]
deliver

column_names = deliver[:-1].columns.tolist() #獲得產品名稱
column_names = column_names[:-1] #移除「訂單編號」（這不是產品名稱）
column_names

# 假设你已经创建了名为deliver的DataFrame

product_columns = column_names

result = {}  # 用于存储结果的字典

for product_column in product_columns:
    filtered_orders = deliver[deliver[product_column] != ""]["訂單編號"]
    result[product_column] = [filtered_orders.tolist()]

result

products_columns = Order_data.columns[6:-2]  # 從第7個欄位開始是產品欄位
Order_data = Order_data[products_columns]
Order_data

# 計算每個產品的總訂購數量
# 將每個列的數值轉換為整數，非數字值設為0
product_quantities = Order_data.apply(pd.to_numeric, errors='coerce').fillna(0)
item_totals = product_quantities.sum()
item_totals = item_totals.to_dict()
item_totals

# format{date} + excel匯出

df = pd.DataFrame(ProductID_data, columns=["供應商", "產品全稱"])
# 提取指定列的数据
selected_columns = df[["供應商", "產品全稱"]]
selected_columns

# Create a new column '合併值' in the DataFrame based on the '產品全稱'
selected_columns['訂購數量'] = selected_columns['產品全稱'].map(item_totals).fillna(0)
# Display the merged DataFrame
selected_columns

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication


# Assuming 'selected_columns' is your DataFrame
grouped = selected_columns.groupby("供應商")

# Load the billing template
template_path = '/content/供應商訂單.xlsx' # gild學長在麻煩您設定成為server的路徑
template_wb = load_workbook(template_path)


# Create a directory to save the individual Excel files
output_directory = f'/content/{today}/' # gild學長在麻煩您設定成為server的路徑
import os
os.makedirs(output_directory, exist_ok=True)

# 設定寄件者的電子郵件地址和密碼作為環境變數
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "coop4s@nthu"

# Iterate through each supplier group and save as separate Excel files
for group_name, group_data in grouped:
    # Create a new workbook for each group_name
    wb = Workbook()
    new_sheet = wb.active

    # Set the sheet title to the group_name
    new_sheet.title = group_name

    # Set up formatting for the new sheet
    new_sheet.column_dimensions['A'].width = 20
    new_sheet.merge_cells('A1:C1')

    font = Font(size=20)
    new_sheet['A1'].font = font

    alignment = Alignment(horizontal='center', vertical='center')
    new_sheet['A1'].alignment = alignment

    new_sheet.merge_cells('A2:C2')
    new_sheet['A2'].alignment = alignment

    fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
    new_sheet['A1'].fill = fill

    new_sheet.column_dimensions['B'].width = 40
    new_sheet.column_dimensions['C'].width = 40

    new_sheet.delete_rows(15, new_sheet.max_row)

    border = Border(
        left=Side(border_style='thick'),
        right=Side(border_style='thick'),
        top=Side(border_style='thick'),
        bottom=Side(border_style='thick')
    )

    for row in new_sheet.iter_rows():
        for cell in row:
            cell.border = border

    # Copy the contents of "供應商訂單" to the new sheet
    source_sheet = template_wb["供應商訂單"]
    for row_index, source_row in enumerate(source_sheet.iter_rows(), start=1):
        for column_index, cell in enumerate(source_row, start=1):
            new_sheet.cell(row=row_index, column=column_index, value=cell.value)

    # Calculate the starting row for data in the new sheet (B15 and C15)
    start_row = 15

    # Retrieve the values from Supplier_data based on group_name
    supplier_info = Supplier_data[Supplier_data["供應商名稱"] == group_name].iloc[0]

    # Write supplier-specific information to the new sheet
    new_sheet.cell(row=8, column=2, value=supplier_info["法人姓名"])
    new_sheet.cell(row=10, column=2, value=str(supplier_info["電話"]))
    new_sheet.cell(row=11, column=2, value=supplier_info["Email"])
    new_sheet.cell(row=9, column=2, value=supplier_info["地址"])
    new_sheet.cell(row=7, column=2, value=group_name)

    # Write the group data to the Excel file starting from B15 and C15
    for index, row in group_data.iterrows():
        new_sheet.cell(row=start_row, column=2, value=row['產品全稱'])
        new_sheet.cell(row=start_row, column=3, value=row['訂購數量'])

        product_name = row['產品全稱']
        if product_name in result.keys():
            values = result[product_name]
        else:
            values = None  # 或者选择一个默认值
        #print(product_name)
        #print(values)
        values = ''.join(map(str, values))
        new_sheet.cell(row=start_row, column=4, value=values)

        start_row += 1



    # Save the current workbook as an individual Excel file
    output_filename = os.path.join(output_directory, f"{today}{group_name}.xlsx")
    wb.save(output_filename)
    #backup_filename = os.path.join(backup_directory, f"{today}{group_name}.xlsx")
    #wb.save(backup_filename)

    # Email settings
    sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
    sender_password = "jwsscmoiffdyukmm"
    recipient_email = str(supplier_info["Email"])  # 確保將電子郵件地址轉換為字符串
    #print(recipient_email)
    subject = "【十里方圓合作社】供應商產品訂購訂單通知。"
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recipient_email
    message["Subject"] = subject

    body = "供應商您好，請檢閱產品訂購數量，若有問題，請您聯絡： 莊棋本 先生，電話：0927433990，E-mail：louie311@gmail.com，再次感謝您的配合。"
    message.attach(MIMEText(body, 'plain'))

    with open(output_filename, 'rb') as attachment:
        part = MIMEApplication(attachment.read(), Name=os.path.basename(output_filename))
    part['Content-Disposition'] = f'attachment; filename="{os.path.basename(output_filename)}"'
    message.attach(part)

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)  # Replace with your SMTP server details
        server.starttls()
        server.login(sender_email, sender_password)

        server.sendmail(sender_email, recipient_email, message.as_string())
        server.quit()

        print(f"Email sent to {recipient_email} with attached Excel file.")
    except smtplib.SMTPException as e:
        print(f"Email sending failed: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


# Remove the default sheet (you may adjust this part)
if 'Sheet' in template_wb.sheetnames:
    template_wb.remove(template_wb['Sheet'])

# No need to save the Excel file here since sheets have already been saved individually.

print(f"Excel files have been created for each supplier and saved to the '{output_directory}' directory.")


files = os.listdir(output_directory)

import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# 載入你的服務帳戶金鑰 JSON 檔案
credentials = service_account.Credentials.from_service_account_file(
    'orderdata-copyto-parttime-bbb4e17511c7.json', scopes=['https://www.googleapis.com/auth/drive']
)

# 建立 Google Drive API 客戶端
drive_service = build('drive', 'v3', credentials=credentials)



# 指定目標文件夾的ID（從你提供的URL中獲得）
folder_id = '1rVzIKeKaMupnTm0CJlTb6Lo8XfZQL2yF'



for file_name in files:
  # 指定要上傳的文件路徑
  file_path = f'/content/{today}/{file_name}'  #Gild學長麻煩修改路徑
  # 創建文件媒體上傳物件
  media = MediaFileUpload(file_path, mimetype='application/octet-stream')
# 上傳文件到目標文件夾
  file_metadata = {
      'name': file_name,
      'parents': [folder_id]
  }

  uploaded_file = drive_service.files().create(
      body=file_metadata,
      media_body=media,
      fields='id'
  ).execute()

  print(f'文件 "{file_name}" 已成功上傳到 Google Drive 文件夾，文件 ID：{uploaded_file.get("id")}')

  import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# 載入你的服務帳戶金鑰 JSON 檔案
credentials = service_account.Credentials.from_service_account_file(
    'orderdata-copyto-parttime-bbb4e17511c7.json', scopes=['https://www.googleapis.com/auth/drive']
)

# 建立 Google Drive API 客戶端
drive_service = build('drive', 'v3', credentials=credentials)



# 指定目標文件夾的ID（從你提供的URL中獲得）
folder_id = '1w0-h2ZLJH7CbkEmgE8FBoEvk9UKlud7H'



for file_name in files:
  # 指定要上傳的文件路徑
  file_path = f'/content/{today}/{file_name}'  #Gild學長麻煩修改路徑
  # 創建文件媒體上傳物件
  media = MediaFileUpload(file_path, mimetype='application/octet-stream')
# 上傳文件到目標文件夾
  file_metadata = {
      'name': file_name,
      'parents': [folder_id]
  }

  uploaded_file = drive_service.files().create(
      body=file_metadata,
      media_body=media,
      fields='id'
  ).execute()

  print(f'文件 "{file_name}" 已成功上傳到 Google Drive 文件夾，文件 ID：{uploaded_file.get("id")}')



ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

KeyboardInterrupt: ignored